In [1]:
import os
import numpy as np
import pandas as pd

from keras.layers import *
from keras import backend as K
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.models import Model,load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

from scipy.misc import toimage,imresize
from skimage import exposure
from PIL import Image
import cv2

Using TensorFlow backend.


### Image generators

In [2]:
spirals_train_folder = '../input/drawings/spiral/training'
spirals_val_folder = '../input/drawings/spiral/testing'
waves_train_folder = '../input/drawings/wave/training'
waves_val_folder = '../input/drawings/wave/testing'

batch_size = 24

# histogram equalizer
def eqz_plz(img):
    return exposure.equalize_hist(img)


spiral_datagen = ImageDataGenerator(rotation_range=360, # they're spirals.
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    brightness_range=(0.5,1.5),
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=eqz_plz,
                                    vertical_flip=True)

wave_datagen = ImageDataGenerator(rotation_range=5,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  brightness_range=(0.5,1.5),
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=eqz_plz,
                                  vertical_flip=True)


spiral_train_generator = spiral_datagen.flow_from_directory(directory=os.path.abspath(spirals_train_folder),
                                                            target_size=(256, 256),
                                                            color_mode="grayscale",
                                                            batch_size=batch_size,
                                                            class_mode="binary",
                                                            shuffle=True,
                                                            seed=666)

spiral_val_generator = spiral_datagen.flow_from_directory(directory=os.path.abspath(spirals_val_folder),
                                                            target_size=(256, 256),
                                                            color_mode="grayscale",
                                                            batch_size=batch_size,
                                                            class_mode="binary",
                                                            shuffle=True,
                                                            seed=710)

wave_train_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(waves_train_folder),
                                                        target_size=(256, 512), # HxW in machine learning, WxH in computer vision
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

wave_val_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(waves_val_folder),
                                                        target_size=(256, 512),
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [3]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=12,min_lr=1e-9,verbose=1)
early_stop = EarlyStopping(monitor='val_loss',patience=16,verbose=1)

## DNN

In [4]:
K.clear_session()

def nopamine_model(mode):
    if (mode == 'spirals') or (mode == 'spiral'):
        input_layer = Input(shape=(256,256,1),name=f'{mode}_input_layer')
    elif (mode == 'waves') or (mode == 'wave'):
        input_layer = Input(shape=(256,512,1),name=f'{mode}_input_layer')

    m1 = Conv2D(256,(5,5),dilation_rate=4,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(input_layer)
    p1 = MaxPool2D((9,9),strides=3)(m1)
    m2 = Conv2D(128,(5,5),dilation_rate=2,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(p1)
    p2 = MaxPool2D((7,7),strides=3)(m2)
    m3 = Conv2D(64,(3,3),kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(p2)
    p3 = MaxPool2D((5,5),strides=2)(m3)
    g1=Dropout(0.2)(p3)
    f1 = Flatten()(g1)
    d1 = Dense(666,activation='relu')(f1)
    g2=Dropout(0.5)(d1)
    d2 = Dense(1,activation='sigmoid')(g2)
    

    
    this_model = Model(input_layer,d2)
    this_model.summary()
    return this_model

### Network for spirals

In [5]:
spiral_model = nopamine_model(mode='spirals') # early stopping epoch 89: val_loss 0.4796, val_acc 0.8274
spiral_model.compile(optimizer=Adam(lr=3.15e-5), loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spirals_input_layer (InputLa (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 256)     6656      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 83, 83, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 83, 128)       819328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)    

In [6]:
result=spiral_model.fit_generator(spiral_train_generator,
                           validation_data=spiral_val_generator,
                           epochs=666,
                           steps_per_epoch=(2000//batch_size),
                           validation_steps=(800//batch_size),
                           callbacks=[reduce_lr,early_stop],
                           verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/666


/opt/conda/lib/python3.6/site-packages/skimage/exposure/exposure.py:124: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


83/83 [==============================] - 40s 483ms/step - loss: 0.9468 - acc: 0.5040 - val_loss: 0.9349 - val_acc: 0.5258
Epoch 2/666
83/83 [==============================] - 30s 366ms/step - loss: 0.9307 - acc: 0.4960 - val_loss: 0.9199 - val_acc: 0.5041
Epoch 3/666
83/83 [==============================] - 31s 376ms/step - loss: 0.9169 - acc: 0.4900 - val_loss: 0.9073 - val_acc: 0.5675
Epoch 4/666
83/83 [==============================] - 30s 367ms/step - loss: 0.9015 - acc: 0.5146 - val_loss: 0.8942 - val_acc: 0.6111
Epoch 5/666
83/83 [==============================] - 31s 375ms/step - loss: 0.8901 - acc: 0.5000 - val_loss: 0.8824 - val_acc: 0.5417
Epoch 6/666
83/83 [==============================] - 30s 360ms/step - loss: 0.8786 - acc: 0.5276 - val_loss: 0.8700 - val_acc: 0.5823
Epoch 7/666
83/83 [==============================] - 31s 372ms/step - loss: 0.8686 - acc: 0.5211 - val_loss: 0.8622 - val_acc: 0.5694
Epoch 8/666
83/83 [==============================] - 30s 358ms/step - loss

In [7]:
# here's how to load/save
spiral_model.save('../nopamine_model_spirals.h5')

### Network for waves

In [8]:
waves_model = nopamine_model(mode='waves')
waves_model.compile(optimizer=Adam(lr=3.15e-5),loss='binary_crossentropy',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
waves_input_layer (InputLaye (None, 256, 512, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 256, 512, 256)     6656      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 83, 168, 256)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 83, 168, 128)      819328    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 26, 54, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 54, 64)        73792     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 11, 25, 64)        0         
__________

In [9]:
waves_model.fit_generator(wave_train_generator,
                          validation_data=wave_val_generator,
                          epochs=666,
                          steps_per_epoch=(2000//batch_size),
                          validation_steps=(800//batch_size),
                          callbacks=[reduce_lr,early_stop],
                          verbose=1)

Epoch 1/666


/opt/conda/lib/python3.6/site-packages/skimage/exposure/exposure.py:124: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


83/83 [==============================] - 62s 753ms/step - loss: 0.9413 - acc: 0.5251 - val_loss: 0.9238 - val_acc: 0.5873
Epoch 2/666
83/83 [==============================] - 55s 668ms/step - loss: 0.9120 - acc: 0.5663 - val_loss: 0.8721 - val_acc: 0.7593
Epoch 3/666
83/83 [==============================] - 55s 668ms/step - loss: 0.8568 - acc: 0.6305 - val_loss: 0.7617 - val_acc: 0.7976
Epoch 4/666
83/83 [==============================] - 55s 665ms/step - loss: 0.7989 - acc: 0.6667 - val_loss: 0.7063 - val_acc: 0.7654
Epoch 5/666
83/83 [==============================] - 55s 664ms/step - loss: 0.7416 - acc: 0.7118 - val_loss: 0.6396 - val_acc: 0.8115
Epoch 6/666
83/83 [==============================] - 55s 664ms/step - loss: 0.7283 - acc: 0.7234 - val_loss: 0.6364 - val_acc: 0.8066
Epoch 7/666
83/83 [==============================] - 55s 666ms/step - loss: 0.6997 - acc: 0.7374 - val_loss: 0.6331 - val_acc: 0.8075
Epoch 8/666
83/83 [==============================] - 55s 668ms/step - loss

In [10]:
waves_model.save('../nopamine_model_waves.h5')